In [41]:
import os

import boto3
import botocore
import configparser
import paramiko

from botocore.exceptions import ClientError

In [42]:
config_aws = configparser.ConfigParser()
config_aws.read_file(open('../aws_credentials.cfg'))

KEY                    = config_aws.get('AWS','KEY')
SECRET                 = config_aws.get('AWS','SECRET')
IAM_ARN                    = config_aws.get('AWS', 'IAM_ARN')

In [43]:
config_instance = configparser.ConfigParser()
config_instance.read_file(open('../aws_setup.cfg'))

AMI_ID                 = config_instance.get('AWS', 'AMI_ID')
KEY_PAIR_NAME          = config_instance.get('AWS', 'KEY_PAIR_NAME') 
EC2_REGION             = config_instance.get('EC2', 'REGION')
EC2_TYPE               = config_instance.get('EC2', 'INSTANCE_TYPE')
EC2_NAME               = config_instance.get('EC2', 'NAME')
VPC_NAME               = config_instance.get('VPC', 'NAME')
VPC_CIDR_BLOCK         = config_instance.get('VPC', 'CIDR_BLOCK')
PUB_SUBNET_NAME        = config_instance.get('VPC', 'PUB_SUBNET_NAME')
PUB_SUBNET_CIDR_BLOCK  = config_instance.get('VPC', 'PUB_SUBNET_CIDR_BLOCK')
PUB_SUBNET_REGION      = config_instance.get('VPC', 'PUB_SUBNET_REGION')
PVR_SUBNET_NAME        = config_instance.get('VPC', 'PVR_SUBNET_NAME')
PVR_SUBNET_CIDR_BLOCK  = config_instance.get('VPC', 'PVR_SUBNET_CIDR_BLOCK')
PVR_SUBNET_REGION      = config_instance.get('VPC', 'PVR_SUBNET_REGION')
RT_NAME                = config_instance.get('ROUTE_TABLE', 'NAME')
IG_NAME                = config_instance.get('IG', 'NAME')


In [7]:
def get_keypair(ec2):
    # call the boto ec2 function to create a key pair
    key_pair = ec2.create_key_pair(KeyName=KEY_PAIR_NAME)
    print("Created a new key pair in AWS.")

    # capture the key and store it in a file
    KeyPairOut = str(key_pair.key_material)

    # create a file to store the key locally
    print("Saving the keypair.")
    key_pair_path = KEY_PAIR_NAME + '.pem'
    with open(key_pair_path, 'w') as f:
        f.write(KeyPairOut)
    os.chmod(key_pair_path, 0o600)
    print("Changed access permission to read-only.")

In [8]:
def create_vpc(ec2):
    # create a new VPC
    print("Creating VPC...")
    vpc = ec2.create_vpc(CidrBlock=VPC_CIDR_BLOCK,
                         TagSpecifications=[{'ResourceType': 'vpc',
                                             'Tags':[{"Key": "Name", 
                                                      "Value": VPC_NAME},
                                                    ]
                                            }])
    
    # wait till available and return VPC ID
    vpc.wait_until_available()
    print(f"VPC {VPC_NAME} is available!")
    return vpc

In [9]:
def create_subnet(ec2, vpc, 
                  subnet_name,
                  subnet_region, 
                  subnet_cidr_block,
                  subnet_type='private'):
    # create a public subnet within the VPC
    print("Creating a "+subnet_type+" subnet...")
    subnet = ec2.create_subnet(
        AvailabilityZone=subnet_region,
    #     AvailabilityZoneId='use2-az2',
        CidrBlock=subnet_cidr_block,
        VpcId=vpc.vpc_id,
        DryRun=False,
        TagSpecifications=[{
            'ResourceType':'subnet',
            'Tags':[{"Key": "Name", "Value": subnet_name},
                   ]
        }])
    print(f"Subnet {subnet_name} is available!")
    return subnet

In [10]:
def stop_instance(ec2_client, instances):
    # get a list of instance ids
    instances_ids = [i.instance_id for i in instances]
    # start the instances
    ec2_client.stop_instances(InstanceIds=instances_ids)
    
    # wait till instance is stopped
    waiter = ec2_client.get_waiter('instance_stopped')
    waiter.wait(InstanceIds=instances_ids)
    print("EC2 instance has stopped!")

In [38]:
def start_instance(ec2_client, instances):
    # get a list of instance ids
    instances_ids = [i.instance_id for i in instances]
    # start the instances
    ec2_client.start_instances(InstanceIds=instances_ids)
    
    # wait till instance is ready
    waiter = ec2_client.get_waiter('instance_running')
    waiter.wait(InstanceIds=instances_ids)
    print("EC2 instance is ready!")

In [12]:
## create a ec2 resource instance
ec2 = boto3.resource('ec2', 
                    region_name = EC2_REGION,
                    aws_access_key_id = KEY,
                    aws_secret_access_key = SECRET)

In [13]:
## create a ec2 client instnace
ec2_client = boto3.client('ec2')

In [15]:
## create a key pair and download them 
get_keypair(ec2)

Created a new key pair in AWS.
Saving the keypair.
Changed access permission to read-only.


In [16]:
## create a VPC instance
vpc = create_vpc(ec2)

Creating VPC...
VPC IMMIGRATE_DEMOGRAPHICS_VPC is available!


In [17]:
# create a public subnet
subnet_pub = create_subnet(ec2=ec2, vpc=vpc, subnet_name=PUB_SUBNET_NAME,
                        subnet_region=PUB_SUBNET_REGION, 
                        subnet_cidr_block=PUB_SUBNET_CIDR_BLOCK,
                        subnet_type='public')
# create a private subnet
subnet_prv = create_subnet(ec2=ec2, vpc=vpc, subnet_name=PVR_SUBNET_NAME,
                        subnet_region=PVR_SUBNET_REGION, 
                        subnet_cidr_block=PVR_SUBNET_CIDR_BLOCK,
                        subnet_type='private')

Creating a public subnet...
Subnet IMMIGRATE_DEMOGRAPHICS_PUB is available!
Creating a private subnet...
Subnet IMMIGRATE_DEMOGRAPHICS_PVR is available!


In [40]:
## create internet gateway
ig = ec2.create_internet_gateway(TagSpecifications=[{
            'ResourceType':'internet-gateway',
            'Tags':[{"Key": "Name", "Value": IG_NAME},
                   ]}])
## attach the internet gateway to the VPC
vpc.attach_internet_gateway(InternetGatewayId = ig.id)

In [ ]:
###### the next 2 steps is to allow SSH connection from outside of VPC to the EC instance

In [20]:
## 1.1 create a new route table for public subnet and
routetable = vpc.create_route_table(
    TagSpecifications=[
        {
            'ResourceType': 'route-table',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': RT_NAME
                },
            ]
        },
    ]
)
## 1.2 create new route, that allows traffic outside of VPC to go to the internet gateway
route = routetable.create_route(DestinationCidrBlock='0.0.0.0/0', GatewayId=ig.id)
## 1.3 attach the new route to the public subnet
routetable.associate_with_subnet(SubnetId=subnet_pub.id)

ec2.RouteTableAssociation(id='rtbassoc-08e9b2a5e7a25762d')

In [21]:
## 2.1 get default security group id
sg = ec2_client.describe_security_groups(Filters=[{'Name': 'vpc-id',
                                                    'Values': [vpc.vpc_id]}
                                                 ])
sg_id = sg['SecurityGroups'][0]['GroupId']
## 2.2 add imbound rule for the security group, allowing SSH connect from all the internet
ec2_client.authorize_security_group_ingress(GroupId = sg_id,
                                        IpPermissions=[
                                        {
                                            'FromPort': 22,
                                            'IpProtocol': 'tcp',
                                            'IpRanges': [
                                                {
                                                    'CidrIp': '0.0.0.0/0',
                                                    'Description': 'SSH access from outside',
                                                },
                                            ],
                                            'ToPort': 22,
                                        },
                                        {
                                            'FromPort': 8080,
                                            'IpProtocol': 'tcp',
                                            'IpRanges': [
                                                {
                                                    'CidrIp': '0.0.0.0/0',
                                                    'Description': 'Apache Airflow',
                                                },
                                            ],
                                            'ToPort': 8080,
                                        }
                                    ],)

{'ResponseMetadata': {'RequestId': '07efd4b2-ba3f-4a68-850b-44b4218d6390',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '07efd4b2-ba3f-4a68-850b-44b4218d6390',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '259',
   'date': 'Wed, 09 Sep 2020 01:36:42 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [45]:
## create EC2 instance
instances = ec2.create_instances(
     ImageId=AMI_ID,
     MinCount=1,
     MaxCount=1,
     InstanceType=EC2_TYPE,
     KeyName=KEY_PAIR_NAME,
     NetworkInterfaces=[{
             'DeviceIndex':0,
             'SubnetId': subnet_pub.subnet_id}],
#               'SubnetId': 'subnet-0ee2d46148e05870f'}],
     TagSpecifications=[{
            'ResourceType':'instance',
            'Tags':[{"Key": "Name", "Value": EC2_NAME}]
            }]
 )

# get instance ids
instances_ids = [i.instance_id for i in instances]

# wait till instance is ready
waiter = ec2_client.get_waiter('instance_running')
waiter.wait(InstanceIds=instances_ids)


In [31]:
## create new EIP and attach it to existing EC2 instance
instance_id = instances[0].instance_id
try:
    allocation = ec2_client.allocate_address(Domain='vpc')
    response = ec2_client.associate_address(AllocationId=allocation['AllocationId'],
                                     InstanceId=instance_id)
    print(response)
except ClientError as e:
    print(e)
print(f"\nThe EIP is: {allocation['PublicIp']}")

{'AssociationId': 'eipassoc-0d45ef4df513ac17f', 'ResponseMetadata': {'RequestId': '0aa0e490-dc46-41eb-896b-cfdcf35a7ac7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0aa0e490-dc46-41eb-896b-cfdcf35a7ac7', 'content-type': 'text/xml;charset=UTF-8', 'content-length': '295', 'date': 'Wed, 09 Sep 2020 01:46:32 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}
The EIP is: 3.12.62.71
